# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 28 2022 10:21AM,242966,10,EYE42977,Eye Pharma Inc,Released
1,Jul 28 2022 10:21AM,242965,16,SSF-8334312,Sartorius Stedim Filters Inc.,Released
2,Jul 28 2022 10:12AM,242963,10,8334277,Beach Products Inc,Released
3,Jul 28 2022 10:12AM,242963,10,8334278,Beach Products Inc,Released
4,Jul 28 2022 10:12AM,242963,10,8334279,Beach Products Inc,Released
5,Jul 28 2022 10:12AM,242963,10,8334280,Beach Products Inc,Released
6,Jul 28 2022 10:12AM,242963,10,8334281,Beach Products Inc,Released
7,Jul 28 2022 10:12AM,242963,10,8334282,Beach Products Inc,Released
8,Jul 28 2022 10:12AM,242963,10,8334283,Beach Products Inc,Released
9,Jul 28 2022 10:12AM,242963,10,8334284,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,242960,Released,7
50,242962,Released,1
51,242963,Released,12
52,242965,Released,1
53,242966,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242960,NaN,NaN,7.0
242962,NaN,NaN,1.0
242963,NaN,NaN,12.0
242965,NaN,NaN,1.0
242966,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242846,19.0,10.0,1.0
242855,0.0,1.0,0.0
242856,0.0,0.0,1.0
242857,0.0,0.0,1.0
242858,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242846,19,10,1
242855,0,1,0
242856,0,0,1
242857,0,0,1
242858,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242846,19,10,1
1,242855,0,1,0
2,242856,0,0,1
3,242857,0,0,1
4,242858,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242846,19,10,1
1,242855,,1,
2,242856,,,1
3,242857,,,1
4,242858,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 28 2022 10:21AM,242966,10,Eye Pharma Inc
1,Jul 28 2022 10:21AM,242965,16,Sartorius Stedim Filters Inc.
2,Jul 28 2022 10:12AM,242963,10,Beach Products Inc
14,Jul 28 2022 10:11AM,242962,10,Emerginnova
15,Jul 28 2022 10:05AM,242960,10,Bio-PRF
22,Jul 28 2022 10:04AM,242950,10,Eywa Pharma Inc.
29,Jul 28 2022 10:03AM,242916,10,Eywa Pharma Inc.
30,Jul 28 2022 9:51AM,242949,19,"AdvaGen Pharma, Ltd"
33,Jul 28 2022 9:45AM,242948,10,"Citieffe, Inc."
34,Jul 28 2022 9:20AM,242947,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 28 2022 10:21AM,242966,10,Eye Pharma Inc,,,1
1,Jul 28 2022 10:21AM,242965,16,Sartorius Stedim Filters Inc.,,,1
2,Jul 28 2022 10:12AM,242963,10,Beach Products Inc,,,12
3,Jul 28 2022 10:11AM,242962,10,Emerginnova,,,1
4,Jul 28 2022 10:05AM,242960,10,Bio-PRF,,,7
5,Jul 28 2022 10:04AM,242950,10,Eywa Pharma Inc.,,,7
6,Jul 28 2022 10:03AM,242916,10,Eywa Pharma Inc.,,,1
7,Jul 28 2022 9:51AM,242949,19,"AdvaGen Pharma, Ltd",,1,2
8,Jul 28 2022 9:45AM,242948,10,"Citieffe, Inc.",,,1
9,Jul 28 2022 9:20AM,242947,10,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 10:21AM,242966,10,Eye Pharma Inc,1,,
1,Jul 28 2022 10:21AM,242965,16,Sartorius Stedim Filters Inc.,1,,
2,Jul 28 2022 10:12AM,242963,10,Beach Products Inc,12,,
3,Jul 28 2022 10:11AM,242962,10,Emerginnova,1,,
4,Jul 28 2022 10:05AM,242960,10,Bio-PRF,7,,
5,Jul 28 2022 10:04AM,242950,10,Eywa Pharma Inc.,7,,
6,Jul 28 2022 10:03AM,242916,10,Eywa Pharma Inc.,1,,
7,Jul 28 2022 9:51AM,242949,19,"AdvaGen Pharma, Ltd",2,1,
8,Jul 28 2022 9:45AM,242948,10,"Citieffe, Inc.",1,,
9,Jul 28 2022 9:20AM,242947,10,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 10:21AM,242966,10,Eye Pharma Inc,1,,
1,Jul 28 2022 10:21AM,242965,16,Sartorius Stedim Filters Inc.,1,,
2,Jul 28 2022 10:12AM,242963,10,Beach Products Inc,12,,
3,Jul 28 2022 10:11AM,242962,10,Emerginnova,1,,
4,Jul 28 2022 10:05AM,242960,10,Bio-PRF,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 10:21AM,242966,10,Eye Pharma Inc,1.0,NaN,NaN
1,Jul 28 2022 10:21AM,242965,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Jul 28 2022 10:12AM,242963,10,Beach Products Inc,12.0,NaN,NaN
3,Jul 28 2022 10:11AM,242962,10,Emerginnova,1.0,NaN,NaN
4,Jul 28 2022 10:05AM,242960,10,Bio-PRF,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 10:21AM,242966,10,Eye Pharma Inc,1.0,0.0,0.0
1,Jul 28 2022 10:21AM,242965,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Jul 28 2022 10:12AM,242963,10,Beach Products Inc,12.0,0.0,0.0
3,Jul 28 2022 10:11AM,242962,10,Emerginnova,1.0,0.0,0.0
4,Jul 28 2022 10:05AM,242960,10,Bio-PRF,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4129721,127.0,32.0,25.0
16,728817,3.0,0.0,0.0
19,1700420,46.0,19.0,2.0
20,1700353,25.0,28.0,1.0
21,971426,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4129721,127.0,32.0,25.0
1,16,728817,3.0,0.0,0.0
2,19,1700420,46.0,19.0,2.0
3,20,1700353,25.0,28.0,1.0
4,21,971426,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,127.0,32.0,25.0
1,16,3.0,0.0,0.0
2,19,46.0,19.0,2.0
3,20,25.0,28.0,1.0
4,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,127.0
1,16,Released,3.0
2,19,Released,46.0
3,20,Released,25.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,25.0,0.0,2.0,1.0,0.0
Executing,32.0,0.0,19.0,28.0,1.0
Released,127.0,3.0,46.0,25.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,25.0,0.0,2.0,1.0,0.0
1,Executing,32.0,0.0,19.0,28.0,1.0
2,Released,127.0,3.0,46.0,25.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,25.0,0.0,2.0,1.0,0.0
1,Executing,32.0,0.0,19.0,28.0,1.0
2,Released,127.0,3.0,46.0,25.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()